# DaySequence1, NightSequence1 Label Dataset

## train_test split

Create annotations_train.csv and annotations_test.csv

In [1]:
# Import Libraries
import os
import numpy as np
import pandas as pd
import glob
import shutil
import time
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2


In [2]:
# Path to Annotation files
DAY_TRAIN_PATH = 'Annotations/Annotations/daySequence1/'
NIGHT_TRAIN_PATH = 'Annotations/Annotations/nightSequence1/'

In [3]:
# Merge all different Annotation Files into a single file
train_day_df = pd.read_csv(os.path.join(DAY_TRAIN_PATH,'frameAnnotationsBOX.csv'),sep=';')
train_day_df['isNight'] = 0

train_night_df = pd.read_csv(os.path.join(NIGHT_TRAIN_PATH,'frameAnnotationsBOX.csv'),sep=';')
train_night_df['isNight'] = 1

df = pd.concat([train_day_df,train_night_df],axis=0)

df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number,isNight
0,dayTest/daySequence1--00000.jpg,stop,706,478,718,500,dayTest/daySequence1/Day1EveningShutter0.00020...,0,dayTest/daySequence1/Day1EveningShutter0.00020...,0,0
1,dayTest/daySequence1--00001.jpg,stop,705,475,720,497,dayTest/daySequence1/Day1EveningShutter0.00020...,1,dayTest/daySequence1/Day1EveningShutter0.00020...,1,0
2,dayTest/daySequence1--00002.jpg,stop,707,476,719,494,dayTest/daySequence1/Day1EveningShutter0.00020...,2,dayTest/daySequence1/Day1EveningShutter0.00020...,2,0
3,dayTest/daySequence1--00005.jpg,stop,708,474,720,492,dayTest/daySequence1/Day1EveningShutter0.00020...,5,dayTest/daySequence1/Day1EveningShutter0.00020...,5,0
4,dayTest/daySequence1--00006.jpg,stop,707,470,722,492,dayTest/daySequence1/Day1EveningShutter0.00020...,6,dayTest/daySequence1/Day1EveningShutter0.00020...,6,0
...,...,...,...,...,...,...,...,...,...,...,...
18509,nightTest/nightSequence1--04860.jpg,go,873,86,1018,206,nightTest/nightSequence1/testSequence1.mp4,4860,nightTest/nightSequence1/testSequence1.mp4,4860,1
18510,nightTest/nightSequence1--04861.jpg,go,946,74,1091,188,nightTest/nightSequence1/testSequence1.mp4,4861,nightTest/nightSequence1/testSequence1.mp4,4861,1
18511,nightTest/nightSequence1--04862.jpg,go,1016,51,1161,177,nightTest/nightSequence1/testSequence1.mp4,4862,nightTest/nightSequence1/testSequence1.mp4,4862,1
18512,nightTest/nightSequence1--04863.jpg,go,1085,42,1245,159,nightTest/nightSequence1/testSequence1.mp4,4863,nightTest/nightSequence1/testSequence1.mp4,4863,1


In [4]:
# Dropping duplicate columns & "Origin file" as we don't need it

df = df.drop(['Origin file','Origin track','Origin track frame number'],axis=1)

In [5]:
# Here Filename (Location of Image) is different -> Change it to appropriate name
# Ex. dayTest/daySequence1--00000.jpg -> daySequence1--00000.jpg

def changeFilename(x):
    filename = x.Filename
    isNight = x.isNight
    
    splitted = filename.split('/')
    clipName = splitted[-1].split('--')[0]
    if isNight:
        return os.path.join(f'{splitted[-1]}')
    
    else:
        return os.path.join(f'{splitted[-1]}')

df['Filename'] = df.apply(changeFilename,axis=1)
df

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin frame number,isNight
0,daySequence1--00000.jpg,stop,706,478,718,500,0,0
1,daySequence1--00001.jpg,stop,705,475,720,497,1,0
2,daySequence1--00002.jpg,stop,707,476,719,494,2,0
3,daySequence1--00005.jpg,stop,708,474,720,492,5,0
4,daySequence1--00006.jpg,stop,707,470,722,492,6,0
...,...,...,...,...,...,...,...,...
18509,nightSequence1--04860.jpg,go,873,86,1018,206,4860,1
18510,nightSequence1--04861.jpg,go,946,74,1091,188,4861,1
18511,nightSequence1--04862.jpg,go,1016,51,1161,177,4862,1
18512,nightSequence1--04863.jpg,go,1085,42,1245,159,4863,1


In [6]:
# We will change annotations to only -> stop (RED), go (GREEN) & warning (YELLOW)

label_to_idx = {'go':1, 'warning':2, 'stop':3}
idx_to_label = {v:k for k,v in label_to_idx.items()}

def changeAnnotation(x):
    if 'go' in x['Annotation tag']:
        return 'go'
    elif 'warning' in x['Annotation tag']:
        return 'warning'
    elif 'stop' in x['Annotation tag']:
        return 'stop'
    
df['Annotation tag'] = df.apply(changeAnnotation,axis=1)

annotation_tags = df['Annotation tag'].unique()
annotation_tags

array(['stop', 'go', 'warning'], dtype=object)

In [7]:
# Total count of labels
df['Annotation tag'].value_counts()

stop       12875
go         12558
warning      981
Name: Annotation tag, dtype: int64

In [8]:
# Changing Column Names
df.columns = ['filename','class','xmin','ymin','xmax','ymax','frame','isNight']

print("Number of Unique Images: ",df.filename.nunique(),'/',df.shape[0])

Number of Unique Images:  8107 / 26414


In [9]:
# get all the image file names from 'frames'
image_files = glob.glob(os.path.join('frames','**/*.jpg'), recursive=True)

print(len(image_files))

9053


In [10]:
# Check duplicates in image files
duped_list = list(set(image_files))

if len(image_files) != len(duped_list):
    print("Duplicates found!")
    duplicates = [item for item in image_files if image_files.count(item) > 1]
    print("Duplicate items:", duplicates)
else:
    print("No duplicates found.")

No duplicates found.


In [11]:
# We will use 80% for training and 20% for test
train_indices = []
test_indices = []

# 20% test
test_indices = random.sample(image_files, int(len(image_files)*0.20))

In [12]:
# Check first 15 image test data
for i in range(0, 15):
    print(test_indices[i])

frames\nightSequence1\nightSequence1\frames\nightSequence1--02631.jpg
frames\daySequence1\daySequence1\frames\daySequence1--00755.jpg
frames\nightSequence1\nightSequence1\frames\nightSequence1--01009.jpg
frames\nightSequence1\nightSequence1\frames\nightSequence1--02172.jpg
frames\daySequence1\daySequence1\frames\daySequence1--01916.jpg
frames\daySequence1\daySequence1\frames\daySequence1--02101.jpg
frames\nightSequence1\nightSequence1\frames\nightSequence1--04138.jpg
frames\daySequence1\daySequence1\frames\daySequence1--02781.jpg
frames\nightSequence1\nightSequence1\frames\nightSequence1--00473.jpg
frames\daySequence1\daySequence1\frames\daySequence1--03124.jpg
frames\daySequence1\daySequence1\frames\daySequence1--01844.jpg
frames\daySequence1\daySequence1\frames\daySequence1--01108.jpg
frames\daySequence1\daySequence1\frames\daySequence1--03550.jpg
frames\nightSequence1\nightSequence1\frames\nightSequence1--00527.jpg
frames\daySequence1\daySequence1\frames\daySequence1--01791.jpg


In [13]:
# Check Length of test data
print("Length of test data : ", len(test_indices))

Length of test data :  1810


In [14]:
# create a new list with the remaining 80% for training
train_indices = [train for train in image_files if train not in test_indices]

# Check Length of train data
print("Length of train data : ", len(train_indices))

Length of train data :  7243


In [15]:
# Copy images to train and test folder
target_train = 'train'
target_test = 'test'

# Train split folder
# specify the source and destination file paths for train
count=0
dst_path_train = os.path.join(os.getcwd(), target_train)
dst_path_train = dst_path_train.replace("\\","/")
print("Destination path of train: ", dst_path_train)

for i in range(0, len(train_indices)):
    src_path_train = os.path.join(os.getcwd(), train_indices[i])
    src_path_train = src_path_train.replace("\\","/")
    print("Source path of train: ", src_path_train)
    if os.path.isfile(train_indices[i]):
        try:
            shutil.copy2(src_path_train, dst_path_train)
            count += 1
        except Exception as e:
            print(f"An error occurred: {e}")
    src_path = None
print("Total Images copied in train folder: ", count)


Destination path of train:  D:/DataScience/Data6400/trafficlightprediction/train
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00000.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00001.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00002.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00003.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00005.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00007.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySe

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00299.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00300.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00304.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00305.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00307.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00308.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00310.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightpre

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00618.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00619.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00620.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00621.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00623.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00625.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00626.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightpre

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00940.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00943.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00944.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00945.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00946.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00947.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00948.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightpre

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01249.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01250.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01251.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01252.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01253.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01254.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01255.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightpre

...

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03609.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03610.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03611.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03612.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03613.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03614.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03615.jpg
Source path of train

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03950.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03951.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03953.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03954.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03955.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03956.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03957.jpg
Source path of train

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04245.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04246.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04247.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04248.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04249.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04250.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04251.jpg
Source path of train

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04574.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04575.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04576.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04577.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04578.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04579.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04580.jpg
Source path of train

Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04914.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04915.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04916.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04918.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04919.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04920.jpg
Source path of train:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04921.jpg
Source path of train

In [16]:
# Test split folder
# specify the source and destination file paths for test
count=0
dst_path_test = os.path.join(os.getcwd(), target_test)
dst_path_test = dst_path_test.replace("\\","/")
print("Destination path of test: ", dst_path_test)

for i in range(0, len(test_indices)):
    src_path_test = os.path.join(os.getcwd(), test_indices[i])
    src_path_test = src_path_test.replace("\\","/")
    print("Source path of test: ", src_path_test)
    if os.path.isfile(test_indices[i]):
        try:
            shutil.copy2(src_path_test, dst_path_test)
            count += 1
        except Exception as e:
            print(f"An error occurred: {e}")
    src_path = None
print("Total Images copied in test folder: ", count)

Destination path of test:  D:/DataScience/Data6400/trafficlightprediction/test
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02631.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00755.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01009.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02172.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01916.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--02101.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00626.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01631.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03634.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01598.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03990.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00579.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00905.jpg
Source path of test:  D:/DataScience/Data6400/trafficligh

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03502.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02699.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01875.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--02481.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03979.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02442.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00340.jpg
Source path of test:  D:/DataScience/Data6400

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02886.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04017.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01794.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00060.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03426.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--02271.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01767.jpg
Source path of test:  D:/DataScience/Data6400

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00043.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01244.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01874.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--02283.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02458.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03601.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01219.jpg
Source path of test:  D:/DataScience/Data6400/traff

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00777.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--02452.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04816.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02393.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00875.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00946.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--03478.jpg
Source path of test:  D:/DataScience/Da

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--04328.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--02841.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00733.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01697.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--01958.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00114.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03767.jpg
Source path of test:  D:/DataScience/Da

Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03482.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--00162.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--00070.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02962.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--01922.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/daySequence1/daySequence1/frames/daySequence1--03070.jpg
Source path of test:  D:/DataScience/Data6400/trafficlightprediction/frames/nightSequence1/nightSequence1/frames/nightSequence1--02680.jpg
Source path of test:  D:/DataScience/Data6400/traff

In [17]:
# Create Annotations Train dataframe
train_indices_anno = []
for i in range(0, len(train_indices)):
    train_indices_anno.append(train_indices[i].split('\\')[-1])   
annotations_train = df[df['filename'].isin(train_indices_anno)]
annotations_train

,filename,class,xmin,ymin,xmax,ymax,frame,isNight
0,daySequence1--00000.jpg,stop,706,478,718,500,0,0
1,daySequence1--00001.jpg,stop,705,475,720,497,1,0
2,daySequence1--00002.jpg,stop,707,476,719,494,2,0
3,daySequence1--00005.jpg,stop,708,474,720,492,5,0
5,daySequence1--00007.jpg,stop,710,470,722,488,7,0
...,...,...,...,...,...,...,...,...
18509,nightSequence1--04860.jpg,go,873,86,1018,206,4860,1
18510,nightSequence1--04861.jpg,go,946,74,1091,188,4861,1
18511,nightSequence1--04862.jpg,go,1016,51,1161,177,4862,1
18512,nightSequence1--04863.jpg,go,1085,42,1245,159,4863,1


In [18]:
# Create Annotations Test dataframe
test_indices_anno = []
for i in range(0, len(test_indices)):
    test_indices_anno.append(test_indices[i].split('\\')[-1])   
annotations_test = df[df['filename'].isin(test_indices_anno)]
annotations_test

,filename,class,xmin,ymin,xmax,ymax,frame,isNight
4,daySequence1--00006.jpg,stop,707,470,722,492,6,0
18,daySequence1--00030.jpg,stop,736,434,748,452,30,0
34,daySequence1--00046.jpg,stop,756,440,768,458,46,0
39,daySequence1--00051.jpg,stop,761,440,773,458,51,0
49,daySequence1--00061.jpg,stop,764,443,776,465,61,0
...,...,...,...,...,...,...,...,...
18477,nightSequence1--04829.jpg,go,1041,58,1093,136,4829,1
18487,nightSequence1--04838.jpg,go,0,126,48,240,4838,1
18490,nightSequence1--04841.jpg,go,33,118,133,250,4841,1
18492,nightSequence1--04843.jpg,go,105,131,191,242,4843,1


In [19]:
# DataFrame to csv
annotations_train.to_csv("D:\\DataScience\\Data6400\\trafficlightprediction\\annotations_train.csv")
annotations_test.to_csv("D:\\DataScience\\Data6400\\trafficlightprediction\\annotations_test.csv")